In [3]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

# display(autentica)


login = str(autentica['login'][10])
senha = str(autentica['senha'][10])

indisponivel = "indisponivel"

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [5]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [6]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [7]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.07.1340,"Café em pó, torrado e moído 100% puro. Embalad...",13,59.15,4.55,07 - GENEROS DE ALIMENTACAO
1,30.07.2,"AÇÚCAR CRISTAL, EMBALAGEM C/ 1 KG.",79,158.00,2.00,07 - GENEROS DE ALIMENTACAO
2,30.11.428,Zinco. Fórmula Química: Zn. Aspecto Físico: pó...,500,7495.00,14.99,11 - MATERIAL QUIMICO
3,30.11.769,"NITRATO DE PRATA p.a, GRAU ACS, MIN 99,0, FRAS...",4,339.20,84.80,11 - MATERIAL QUIMICO
4,30.11.834,"PERÓXIDO DE HIDROGÊNIO, líquido incolor, compo...",3,48.00,16.00,11 - MATERIAL QUIMICO
...,...,...,...,...,...,...
601,30.44.68,"Fita adesiva, material PVC, tipo auto-adesiva ...",10,556.80,55.68,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
602,30.59.23,Caneta ecológica personalizada - click(ponta r...,40,56.40,1.41,59 - MATERIAL PARA DIVULGAÇÃO
603,30.59.24,Bolsa para evento - ecobag. Sacola de evento. ...,80,356.00,4.45,59 - MATERIAL PARA DIVULGAÇÃO
604,30.59.25,Squeeze em polietileno com capacidade de 500ml...,72,210.24,2.92,59 - MATERIAL PARA DIVULGAÇÃO
